In [6]:
import os
import mlflow
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

# =========================================================
# 1. CONFIGURACIÓN
# =========================================================
if not load_dotenv(dotenv_path='../.env'):
    load_dotenv(dotenv_path='.env')

NEO4J_URI = os.getenv("NEO4J_URI")
# Fix para local vs docker
if "neo4j" in NEO4J_URI and "localhost" not in NEO4J_URI:
    NEO4J_URI = NEO4J_URI.replace("neo4j", "localhost")

NEO4J_AUTH = (os.getenv("NEO4J_USER", "neo4j"), os.getenv("NEO4J_PASSWORD"))
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

tracking_uri = os.getenv("MLFLOW_TRACKING_URI_LOCAL") or "http://localhost:5000"
mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("Agente_Con_Stock_Feedback")

print(f"✅ Conectado a Neo4j: {NEO4J_URI}")
print("⏳ Cargando embeddings...")
embedder = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Listo.")

# =========================================================
# 2. HERRAMIENTA DE BÚSQUEDA (AHORA CON STOCK)
# =========================================================
@tool
def consultar_catalogo(intencion_usuario: str):
    """
    Busca productos, precios y DISPONIBILIDAD EN TIENDAS.
    """
    print(f"   [TOOL] 🔍 Buscando: '{intencion_usuario}'")
    query_vector = embedder.encode(intencion_usuario).tolist()
    
    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)
    
    # --- QUERY ACTUALIZADA PARA VER TIENDAS Y STOCK ---
    cypher = """
    CALL db.index.vector.queryNodes('productos_embeddings', 3, $vector)
    YIELD node AS p, score
    WHERE score > 0.5
    
    // 1. Ver accesorios (Cross-selling)
    OPTIONAL MATCH (p)-[:COMPATIBLE_CON]->(acc:Producto)
    
    // 2. Ver Disponibilidad (Stock en Tiendas)
    OPTIONAL MATCH (t:Tienda)-[s:TIENE_STOCK]->(p)
    WHERE s.cantidad > 0
    
    RETURN 
        p.nombre AS producto, 
        p.precio AS precio, 
        p.descripcion AS desc,
        collect(DISTINCT acc.nombre) AS accesorios,
        // Agrupar stock: "Tienda Norte: 5"
        collect(DISTINCT t.nombre + ': ' + toString(s.cantidad) + ' unid.') AS stock_info
    """
    
    try:
        with driver.session() as session:
            result = session.run(cypher, vector=query_vector)
            data = [dict(record) for record in result]
            
        if not data: return "No se encontraron productos."
        
        txt = "Inventario:\n"
        for item in data:
            txt += f"- {item['producto']} (${item['precio']})\n"
            txt += f"  Desc: {item['desc']}\n"
            
            # Mostrar Stock
            if item['stock_info']:
                txt += f"  ✅ Disponibilidad: {', '.join(item['stock_info'])}\n"
            else:
                txt += f"  ❌ Agotado en todas las tiendas.\n"
                
            if item['accesorios']:
                txt += f"  💡 Accesorios: {', '.join(item['accesorios'])}\n"
            txt += "\n"
        return txt
        
    except Exception as e:
        return f"Error DB: {e}"
    finally:
        driver.close()

# =========================================================
# 3. EL AGENTE
# =========================================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools([consultar_catalogo])

def agente_conversacional(query):
    messages = [
        SystemMessage(content="Eres un vendedor. Si preguntan por productos, SIEMPRE indica en qué tiendas hay stock disponible."),
        HumanMessage(content=query)
    ]
    ai_msg = llm_with_tools.invoke(messages)
    messages.append(ai_msg)
    
    if ai_msg.tool_calls:
        for tool_call in ai_msg.tool_calls:
            res = consultar_catalogo.invoke(tool_call["args"])
            messages.append(ToolMessage(content=str(res), tool_call_id=tool_call["id"]))
        return llm_with_tools.invoke(messages).content
    return ai_msg.content

# =========================================================
# 4. PRUEBA INTERACTIVA CON FEEDBACK MANUAL
# =========================================================
def prueba_interactiva():
    pregunta = input("\n👤 ¿Qué estás buscando?: ") # Ej: Laptop barata
    
    print("\n🚀 Consultando al Agente...")
    with mlflow.start_run():
        mlflow.log_param("query", pregunta)
        
        # 1. Obtener respuesta
        respuesta = agente_conversacional(pregunta)
        print(f"\n🤖 AGENTE: {respuesta}")
        mlflow.log_text(respuesta, "respuesta_agente.txt")
        
        # 2. Solicitar Feedback Manual
        print("\n" + "="*30)
        es_correcto = input("Evaluación: ¿Respondió bien? (s/n): ").lower()
        
        score = 1 if es_correcto == 's' else 0
        mlflow.log_metric("satisfaccion_usuario", score)
        
        if score == 0:
            comentario = input("¿Qué salió mal? (ej: 'No diste stock'): ")
            print("📝 Guardando queja en MLflow...")
            mlflow.log_text(comentario, "feedback_negativo.txt")
            print("❌ Feedback Negativo Registrado.")
        else:
            print("✅ Feedback Positivo Registrado.")

if __name__ == "__main__":
    # Ejecuta esto cuantas veces quieras probar
    while True:
        prueba_interactiva()
        if input("\n¿Otra prueba? (s/n): ").lower() != 's':
            break

✅ Conectado a Neo4j: bolt://localhost:7687
⏳ Cargando embeddings...


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 759.00it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Listo.

🚀 Consultando al Agente...
   [TOOL] 🔍 Buscando: 'buscar laptop ligera'
   [TOOL] 🔍 Buscando: 'buscar accesorios para laptop'

🤖 AGENTE: Aquí tienes algunas opciones de laptops ligeras y sus accesorios disponibles:

### Laptops Ligeras

1. **MacBook Air M2**
   - **Precio:** $1200
   - **Descripción:** Laptop ligera Apple con chip M2 de 13 pulgadas
   - **Disponibilidad:**
     - Venta Online: 2 unidades
     - Sucursal Norte: 8 unidades
     - Tienda Central: 7 unidades
   - **Accesorios Disponibles:**
     - Cargador Anker 100W
     - Sony WH-1000XM5
     - Monitor LG Ultrawide
     - Logitech MX Master 3S

2. **Dell XPS 13**
   - **Precio:** $1100
   - **Descripción:** Ultrabook Windows con pantalla InfinityEdge
   - **Disponibilidad:**
     - Venta Online: 2 unidades
     - Sucursal Norte: 4 unidades
     - Tienda Central: 2 unidades
   - **Accesorios Disponibles:**
     - Cargador Anker 100W
     - Monitor LG Ultrawide
     - Logitech MX Master 3S

### Accesorios para La